This notebook is used to back test the model we developed to predict stock trend predictions.

### Imports

In [1]:
%matplotlib inline
import yfinance as yf
import backtrader as bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import json

from fetch_stock_data import get_stock_data
from normalize_data import standardize_stock_data
from trendformer import transformer_model, feature_size, d_model, seq_length, dropout_rate, num_heads, num_layers, dff

### Pull market data for model -30 days from start day of simulation

In [2]:
symbol = 'MSFT'
normalized_params_path = 'G:/StockData/normalized_master_datasets/abs_normalized_parameters.json'

start_date = (datetime.now() - timedelta(days=150)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

stock_data = get_stock_data(start_date, end_date, {symbol:[]})

with open(normalized_params_path, 'r') as file:
    normalized_params = json.load(file)

stock_data, _ = standardize_stock_data(stock_data, normalized_params)

dates = stock_data[symbol]

features_to_keep = ['week_day', 'Open', 'High', 'Low', 'Close', 'Volume','SMA_30', 'SMA_40', 'EMA_10', 'EMA_30', 'MACDh_12_26_9', 'RSI_10', 'BBP_5_2.0', 'avgTradingVolume', 'STOCHk_14_3_3', '52_week_high', '52_week_low', 'NASDAQ_Close', 'STOCHd_14_3_3', 'ADX_14', 'DMP_14', 'DMN_14', 'ISA_9', 'ISB_26', 'ITS_9', 'IKS_26', 'PSAR_combined']

stock_data = {date: {key: value for key, value in sorted(data.items()) if key in features_to_keep} for date, data in dates.items()}
print(stock_data)


Fetching data for MSFT...
{'2023-07-24': {'52_week_high': 4.006354559001641, '52_week_low': 4.514569815460835, 'ADX_14': 0.28803225138149147, 'BBP_5_2.0': -0.7651587779974319, 'Close': 4.769525344512495, 'DMN_14': -0.3715958411571963, 'DMP_14': 0.7257593978416538, 'EMA_10': 4.801790661960604, 'EMA_30': 4.746079545779857, 'High': 4.720088616145754, 'IKS_26': 4.878213603334439, 'ISA_9': 4.633270554512545, 'ISB_26': 4.377919193279372, 'ITS_9': 4.900191598845316, 'Low': 4.80812127084846, 'MACDh_12_26_9': -0.007072833328030852, 'NASDAQ_Close': 1.5187466846902926, 'Open': 4.7820469733915125, 'PSAR_combined': 4.623938548781509, 'RSI_10': 0.09306361591071917, 'SMA_30': 4.756866087685692, 'SMA_40': 4.7455026075616304, 'STOCHd_14_3_3': 0.056994206210691664, 'STOCHk_14_3_3': -0.29123227735357926, 'Volume': -0.1922293296942765, 'avgTradingVolume': 4.799494763958018, 'week_day': -1.447715110949788}, '2023-07-25': {'52_week_high': 4.006354559001641, '52_week_low': 4.514569815460835, 'ADX_14': 0.2859

### We need to pull intraday market data into a CSV for backtrader

In [3]:
def fetch_and_save_intraday_data(symbol, interval='1d'):
    # Create the subdirectory if it doesn't exist
    if not os.path.exists('historical_intraday_market_data'):
        os.makedirs('historical_intraday_market_data')
    
    # Calculate start and end dates for the last 60 days
    end = datetime.now()
    start = end - timedelta(days=59)

    # Fetch the intraday market data
    data = yf.download(symbol, start=start, end=end, interval=interval)
    
    # Save the data to a CSV file
    filename = f'historical_intraday_market_data/{symbol}_{interval}.csv'
    data.to_csv(filename)
    
    print(f'Intraday market data for {symbol} saved to {filename}')

fetch_and_save_intraday_data(symbol, interval='2m')


[*********************100%%**********************]  1 of 1 completed
Intraday market data for MSFT saved to historical_intraday_market_data/MSFT_2m.csv


### Creating model and load weights


In [4]:
# Create the transformer model
model = transformer_model(seq_length, feature_size, d_model, num_heads, dff, num_layers, dropout_rate)

# Load the weights
model.load_weights('model_weights\model_weights.h5')
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 27)]     0           []                               
                                                                                                  
 dense (Dense)                  (None, 30, 512)      14336       ['input_1[0][0]']                
                                                                                                  
 tf.__operators__.add_1 (TFOpLa  (None, 30, 512)     0           ['dense[0][0]']                  
 mbda)                                                                                            
                                                                                                  
 encoder_layer_1 (Functional)   (None, None, 512)    9453568     ['tf.__operators__.add_1[0][0

In [13]:
def backtest(csv_file, strategy):
    # Read the CSV file into a pandas dataframe
    df = pd.read_csv(csv_file)

    # Convert the 'Datetime' column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Create a backtrader data feed
    data = bt.feeds.PandasData(dataname=df, datetime='Datetime', open='Open', high='High', low='Low', close='Close', volume='Volume')

    # Create a backtrader Cerebro instance
    cerebro = bt.Cerebro()

    # Add the data feed to the Cerebro instance
    cerebro.adddata(data)

    # Add the SMA strategy to the Cerebro instance
    cerebro.addstrategy(strategy, model)

    # Set the initial capital
    cerebro.broker.setcash(100000)

    # Run the backtest
    cerebro.run()

    # Print the final portfolio value
    print('Final Portfolio Value:', cerebro.broker.getvalue())

### Trendformer Strategy

In [14]:
class TrendformerStrategy(bt.Strategy):
        def __init__(self, model):
            self.model = model
            self.current_date = None
            self.past_30_days = {}
            self.buy_price = None
            self.is_open = False
            self.profit_count = 0
            self.loss_count = 0
            self.prediction = None
            self.last_day_sma_30 = None
            self.size = None

        def next(self):
            # Get the current iteration date
            current_date = self.data.datetime.datetime()

            # Convert current date to string in format YYYY-MM-DD
            current_date_str = current_date.strftime('%Y-%m-%d')

            if self.current_date != current_date_str and current_date_str in stock_data:
                self.current_date = current_date_str

                # Get the keys (dates) of the stock data as a list
                dates = list(stock_data.keys())

                # Find the index of the current date
                current_date_index = dates.index(current_date_str)

                # Get the 30 dates before the current date, excluding the current date
                last_30_dates = dates[max(0, current_date_index - 30):current_date_index]

                # Get the stock data for the last 30 dates
                past_30_days = {date: stock_data[date] for date in last_30_dates}

                self.past_30_days = past_30_days

                # Get the SMA_30 value of the last day in the past_30_days array
                self.last_day_sma_30 = self.past_30_days[last_30_dates[-1]]['SMA_30']
                # print(f"SMA_30 value of the last day: {last_day_sma_30}")

                 # Convert the past_30_days data to a numpy array
                past_30_days_data = np.array([list(day.values()) for day in self.past_30_days.values()])

                # Add an extra dimension to represent the batch size
                past_30_days_data = np.expand_dims(past_30_days_data, axis=0)

                # Now you can pass this data into your model
                self.prediction = self.model.predict(past_30_days_data, verbose=0)
                self.prediction = self.prediction[0][0]

                # print(prediction)
                
                # If the prediction is above the sma 30 value buy
                if self.prediction > self.last_day_sma_30 and self.is_open == False:

                    # Get the current price
                    self.buy_price = self.data.close[0]

                    # Get the current cash
                    cash = self.broker.getcash()

                    # Calculate the number of shares to buy
                    self.size = cash // self.buy_price

                    self.buy(size=self.size)
                    self.is_open = True
                    print(f"Buy at {self.data.close[0]} because prediction is above SMA_30 value")
                
                
            # print(f"Prediction: {self.prediction}")
            # print(f"Last day SMA_30: {self.last_day_sma_30}")
            # We want to sell if the price is 1% above the buy price. Otherwise we just wait it out and hope it goes up.
            if self.prediction < self.last_day_sma_30 and self.is_open == True and self.data.close[0] > self.buy_price:
                self.sell()
                self.is_open = False
                self.profit_count += 1
                print(f"Sell at {self.data.close[0]} to lock in profit. Prediction is below SMA_30 value and we are above buy price")
                # Print current portfol value
                print(f"Current portfolio value: {self.broker.getvalue()}")
                
            # Tick by tick sell logic
            if self.is_open == True:
                # Stop loss
                if self.data.close[0] < self.buy_price * 0.95:
                    self.sell(size=self.size)
                    self.is_open = False
                    print(f"Sell at {self.data.close[0]} to stop loss")
                    print(f"Current portfolio value: {self.broker.getvalue()}")
                    self.loss_count += 1
                # Sell if price is 5% above buy price
                elif self.data.close[0] > self.buy_price * 1.05:
                    self.sell(size=self.size)
                    self.is_open = False
                    print(f"Sell at {self.data.close[0]} to lock in profit")
                    print(f"Current portfolio value: {self.broker.getvalue()}")
                    self.profit_count += 1
            # print(f"Profit count: {self.profit_count}")
            # print(f"Loss count: {self.loss_count}")
                    
backtest(f'historical_intraday_market_data/{symbol}_2m.csv', TrendformerStrategy)

C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Buy at 349.3099975585937 because prediction is above SMA_30 value
Sell at 366.7850036621094 to lock in profit
Current portfolio value: 105003.56860351562
Buy at 367.7200012207031 because prediction is above SMA_30 value
Final Portfolio Value: 106911.65069580078


### SMA Strategy

In [15]:
class SMAStrategy(bt.Strategy):
    params = (
        ('sma_period', 30),  # Period for the SMA
    )

    def __init__(self, model):
        self.sma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.sma_period
        )

    def next(self):
        if self.data.close[0] > self.sma[0]:
            self.buy()
        elif self.data.close[0] < self.sma[0]:
            self.sell()

backtest(f'historical_intraday_market_data/{symbol}_2m.csv', SMAStrategy)

C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 102628.76974487305


### Buy and Hold Strategy

In [17]:
class BuyAndHoldStrategy(bt.Strategy):
    def __init__(self, model):
        pass

    def next(self):
        if not self.position:
            self.buy()

backtest('historical_intraday_market_data/DIS_2m.csv', BuyAndHoldStrategy)

C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 100008.48000335693


### Moving Average Crossover Strategy

In [19]:
class MovingAverageCrossoverStrategy(bt.Strategy):
    params = (
        ('short_period', 50),  # Period for the short-term moving average
        ('long_period', 200),  # Period for the long-term moving average
    )

    def __init__(self, model):
        self.short_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.short_period
        )
        self.long_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.long_period
        )

    def next(self):
        if self.short_ma[0] > self.long_ma[0] and self.short_ma[-1] < self.long_ma[-1]:
            self.buy()
        elif self.short_ma[0] < self.long_ma[0] and self.short_ma[-1] > self.long_ma[-1]:
            self.sell()

backtest(f'historical_intraday_market_data/{symbol}_2m.csv', MovingAverageCrossoverStrategy)

C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 99977.68508911133


### Relative Strength Index (RSI) Strategy

In [20]:
class RSIStrategy(bt.Strategy):
    params = (
        ('rsi_period', 14),  # Period for the RSI
        ('rsi_buy_threshold', 30),  # RSI threshold for buying
        ('rsi_sell_threshold', 70),  # RSI threshold for selling
    )

    def __init__(self, model):
        self.rsi = bt.indicators.RelativeStrengthIndex(
            self.data.close, period=self.params.rsi_period
        )

    def next(self):
        if self.rsi[0] < self.params.rsi_buy_threshold:
            self.buy()
        elif self.rsi[0] > self.params.rsi_sell_threshold:
            self.sell()

backtest(f'historical_intraday_market_data/{symbol}_2m.csv', RSIStrategy)


C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 100093.44882202148


### MACD Strategy

In [21]:
class MACDStrategy(bt.Strategy):
    def __init__(self, model):
        self.macd = bt.indicators.MACD(self.data.close)

    def next(self):
        if self.macd.macd[0] > self.macd.signal[0]:
            self.buy()
        elif self.macd.macd[0] < self.macd.signal[0]:
            self.sell()

backtest(f'historical_intraday_market_data/{symbol}_2m.csv', MACDStrategy)

C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 97343.0241394043


### Bollinger Bands Strategy

In [22]:
class BollingerBandsStrategy(bt.Strategy):
    params = (
        ('bb_period', 20),  # Period for the Bollinger Bands
        ('bb_devfactor', 2),  # Standard deviation factor for the Bollinger Bands
    )

    def __init__(self, model):
        self.bbands = bt.indicators.BollingerBands(
            self.data.close, period=self.params.bb_period, devfactor=self.params.bb_devfactor
        )

    def next(self):
        if self.data.close[0] < self.bbands.lines.bot[0]:
            self.buy()
        elif self.data.close[0] > self.bbands.lines.top[0]:
            self.sell()

backtest(f'historical_intraday_market_data/{symbol}_2m.csv', BollingerBandsStrategy)


C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 100000.41595458984


### Custom Strategy using multiple technical indicators

In [23]:
class CustomStrategy(bt.Strategy):
    def __init__(self, model):
        self.model = model
        self.indicators = []

        # Add your technical indicators here
        self.indicators.append(bt.indicators.SimpleMovingAverage(self.data.close, period=10))
        self.indicators.append(bt.indicators.ExponentialMovingAverage(self.data.close, period=20))
        self.indicators.append(bt.indicators.RSI(self.data.close, period=14))
        self.indicators.append(bt.indicators.MACD(self.data.close))
        self.indicators.append(bt.indicators.BollingerBands(self.data.close))

    def next(self):
        buy_signals = 0
        sell_signals = 0

        # Check the signals of each indicator
        for indicator in self.indicators:
            if indicator[0] > indicator[-1]:
                buy_signals += 1
            elif indicator[0] < indicator[-1]:
                sell_signals += 1

        # Buy if 3 of the 5 indicators signal buy
        if buy_signals >= 3:
            self.buy()

        # Sell if 3 of the 5 indicators signal sell
        elif sell_signals >= 3:
            self.sell()

backtest(f'historical_intraday_market_data/{symbol}_2m.csv', CustomStrategy)

C:\Users\Aman\AppData\Local\Temp\ipykernel_7212\1240526042.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Datetime'] = pd.to_datetime(df['Datetime'])


Final Portfolio Value: 101810.52774047852
